# Recurrent Neural Network Ejemplo

Construir una red neural recurrente (LSTM) con TensorFlow.

## RNN

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" alt="nn" style="width: 600px;"/>

Referencias:
- [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf), Sepp Hochreiter & Jurgen Schmidhuber, Neural Computation 9(8): 1735-1780, 1997.

## MNIST Descripción general del conjunto de datos

Este ejemplo utiliza dígitos manuscritos MNIST. El conjunto de datos contiene 60.000 ejemplos de formación y 10.000 ejemplos de pruebas. Los dígitos han sido normalizados y centrados en una imagen de tamaño fijo (28x28 píxeles) con valores de 0 a 1. Para simplificar, cada imagen ha sido aplanada y convertida a una matriz numérica 1-D de 784 características (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

To classify images using a recurrent neural network, we consider every image row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then handle 28 sequences of 28 timesteps for every sample.

Más información: http://yann.lecun.com/exdb/mnist/

In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

# Importar datos MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Entrenar Parametros
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Network Parametros
num_input = 28 # Entrada de datos MNIST (forma de la imagen: 28*28)
timesteps = 28 # pasos temporales
num_hidden = 128 # capa oculta número de características
num_classes = 10 # Clases totales del MNIST (0-9 dígitos)

# tf Entrada de gráficos
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [3]:
# Definir pesos
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [4]:
def RNN(x, weights, biases):

    # Preparar la forma de los datos para que coincida con los requisitos de la función `rnn
    # Forma actual de entrada de datos: (batch_size, timesteps, n_input)
    # Forma requerida: `pasos de tiempo' tensores lista de formas (batch_size, n_input)

    # Desapilar para obtener una lista de tensores de forma de'pasos de tiempo' (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define una célula lstm con tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Obtener la salida de la célula del lstm
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Activación lineal, utilizando la última salida del bucle interno rnn
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [5]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define pérdida y optimizador
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluar el modelo (con logits de prueba, para que la salida sea desactivada)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Inicializar las variables (es decir, asignar su valor por defecto)
init = tf.global_variables_initializer()

In [6]:
# Iniciar Entrenamiento
with tf.Session() as sess:

    # Ejecutar el inicializador
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reestructurar los datos para obtener 28 seq de 28 elementos
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Optimización de la ejecución (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calcular la pérdida y la precisión de los lotes
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calcule la precisión de las imágenes de prueba de 128 mnist
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.6268, Training Accuracy= 0.102
Step 200, Minibatch Loss= 2.0722, Training Accuracy= 0.328
Step 400, Minibatch Loss= 1.9181, Training Accuracy= 0.336
Step 600, Minibatch Loss= 1.8858, Training Accuracy= 0.336
Step 800, Minibatch Loss= 1.7022, Training Accuracy= 0.422
Step 1000, Minibatch Loss= 1.6365, Training Accuracy= 0.477
Step 1200, Minibatch Loss= 1.6691, Training Accuracy= 0.516
Step 1400, Minibatch Loss= 1.4626, Training Accuracy= 0.547
Step 1600, Minibatch Loss= 1.4707, Training Accuracy= 0.539
Step 1800, Minibatch Loss= 1.4087, Training Accuracy= 0.570
Step 2000, Minibatch Loss= 1.3033, Training Accuracy= 0.570
Step 2200, Minibatch Loss= 1.3773, Training Accuracy= 0.508
Step 2400, Minibatch Loss= 1.3092, Training Accuracy= 0.570
Step 2600, Minibatch Loss= 1.2272, Training Accuracy= 0.609
Step 2800, Minibatch Loss= 1.1827, Training Accuracy= 0.633
Step 3000, Minibatch Loss= 1.0453, Training Accuracy= 0.641
Step 3200, Minibatch Loss= 1.0400, Training Acc